In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import os

_current_fname = Path(locals()["__vsc_ipynb_file__"])
os.chdir(_current_fname.parent)

In [3]:
import copick
import multiprocess as mp
import numpy as np

mp.set_start_method("spawn")


In [8]:
def run_in_separate_process(
    run_id,
    out_dir,
    device=0,
    min_mask_area=2000,
):
    def overlap(img1, img2):
        ints = np.logical_and(img1, img2).sum()
        return ints / min(img1.sum(), img2.sum())

    print("Using device: ", device)
    import os

    os.environ["CUDA_VISIBLE_DEVICES"] = str(device)
    from pathlib import Path

    import copick
    import joblib
    import my_sam2_utils as utils
    import numpy as np
    import skimage
    import torch

    # use bfloat16 for the entire notebook.
    torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

    if torch.cuda.get_device_properties(0).major >= 8:
        # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

    # select the device for computation
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"using device: {device}")

    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

    from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
    from sam2.build_sam import build_sam2, build_sam2_tomogram_predictor

    copick_config = (
        "/hpc/projects/group.czii/krios1.processing/copick/24jul29c/run002/copick_config.json"
    )

    # Open Copick Project and Query All Available Runs
    root = copick.from_file(copick_config)
    run = root.get_run(run_id)
    vol = utils.get_tomogram(run)

    print(f"Preprocessing the volume for {run_id}")

    vol_mean = vol.mean()
    vol_std = vol.std()
    vol_normed = np.clip(vol, vol_mean - 5 * vol_std, vol_mean + 5 * vol_std)
    vol_normed = (vol_normed - vol_normed.min()) / (vol_normed.max() - vol_normed.min())
    vol_normed = vol_normed * 2 - 1

    zSlice = vol.shape[0] // 2  # int(points.mean(axis=0)[0])
    deltaZ = 50  # int((np.max(points[:, 0]) - np.min(points[:, 0]))) * 2

    min_mask_area = 2000

    #
    image = utils.project_tomogram(vol_normed, zSlice, deltaZ)
    image = (image - image.min()) / (image.max() - image.min()) * 2 - 1

    # Extend From Grayscale to RGB
    image = np.repeat(image[..., None], 3, axis=2)

    sam2_checkpoint = (
        "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/checkpoints/sam2_hiera_large.pt"
    )
    model_cfg = "sam2_hiera_l.yaml"

    sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=True)
    # mask_generator = SAM2AutomaticMaskGenerator(sam2)
    mask_generator = SAM2AutomaticMaskGenerator(
        model=sam2,
        points_per_side=32,
        points_per_batch=64,
        pred_iou_thresh=0.7,
        stability_score_thresh=0.92,
        stability_score_offset=0.7,
        crop_n_layers=1,
        box_nms_thresh=0.5,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=2000,
        use_m2m=True,
        multimask_output=True,
        max_rel_box_size=0.8,
        min_rel_box_size=0.15,
    )

    print("Running automatic segmentation for projection")
    masks = mask_generator.generate(image)

    # Filter masks based on the area key and on side length
    masks_filtered = []
    for mask in masks:
        _x1, _y1, w, h = mask["bbox"]
        if (
            (mask["area"] < min_mask_area)
            or (w > 0.8 * image.shape[1])
            or (h > 0.8 * image.shape[0])
        ):
            continue
        masks_filtered.append(mask)
    masks = masks_filtered
    flags = {"overlap": False}
    # Filter based on oveerlap
    masks_to_remove = []

    # for _i1, _mask1 in enumerate(masks):
    #    _x1, _y1, w, h = _mask1["bbox"]
    #    for _i2, _mask2 in enumerate(masks):
    #        if _i1 >= _i2:
    #            continue
    #        overlap_score = overlap(_mask1["segmentation"], _mask2["segmentation"])
    #
    #        if overlap_score > 0.9:
    #            flags["overlap"] = True
    #            if _mask1["area"] > _mask2["area"]:
    #                masks_to_remove.append(_i1)
    #            else:
    #                masks_to_remove.append(_i2)
    # masks_to_remove = np.unique(masks_to_remove)
    # masks = [masks[i] for i in range(len(masks)) if i not in masks_to_remove]
    masks = sorted(masks, key=lambda x: x["area"])

    mask_outputs = {"masks": masks, "flags": flags}
    print("Saving the masks")
    # out_dir = Path("cryosam_outputs_no_overlap")
    Path(out_dir).mkdir(exist_ok=True)
    joblib.dump(mask_outputs, out_dir / f"auto_masks_{run_id}.pkl")

    print("Running automatic segmentation for volume")
    model_cfg_2 = "sam2_hiera_l_custom.yaml"
    predictor = build_sam2_tomogram_predictor(model_cfg_2, sam2_checkpoint, device=device)

    inference_state = predictor.init_state(vol_normed)
    predictor.reset_state(inference_state)

    ann_frame_idx = zSlice

    for ii in range(len(masks)):
        # sam_points = (auto_points[ii, :] * scale).reshape(1, 2)
        ann_obj_id = ii + 1

        _, out_obj_ids, out_mask_logits = predictor.add_new_mask(
            inference_state=inference_state,
            frame_idx=ann_frame_idx,
            obj_id=ann_obj_id,
            mask=masks[ii]["segmentation"],
        )

    video_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(
        inference_state, start_frame_idx=zSlice, reverse=False
    ):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

        # Pull out Masks for Multiple Classes
    nMasks = len(video_segments[zSlice])
    vol_mask = np.zeros([vol.shape[0], vol.shape[1], vol.shape[2]], dtype=np.uint8)
    for out_frame_idx in list(video_segments):
        for jj in range(nMasks):
            resized_mask = skimage.transform.resize(
                video_segments[out_frame_idx][jj + 1][0,],
                (vol.shape[1], vol.shape[2]),
                anti_aliasing=False,
            )
            mask_update = resized_mask > 0
            vol_mask[out_frame_idx, :, :][mask_update] = jj + 1

    # run propagation throughout the video and collect the results in a dict
    video_segments = {}  # video_segments contains the per-frame segmentation results
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(
        inference_state, start_frame_idx=zSlice, reverse=True
    ):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    # Pull out Masks for Multiple Classes
    for out_frame_idx in list(video_segments):
        for jj in range(nMasks):
            resized_mask = skimage.transform.resize(
                video_segments[out_frame_idx][jj + 1][0,],
                (vol.shape[1], vol.shape[2]),
                anti_aliasing=False,
            )
            mask_update = resized_mask > 0
            vol_mask[out_frame_idx, :, :][mask_update] = jj + 1

    # Save the mask and the volume
    print("Saving the masks and the volume")

    joblib.dump(vol_mask, out_dir / f"vol_mask_{run_id}.pkl")
    return masks, vol_mask

In [9]:
copick_config = (
    "/hpc/projects/group.czii/krios1.processing/copick/24jul29c/run002/copick_config.json"
)

# Open Copick Project and Query All Available Runs
root = copick.from_file(copick_config)
run_ids = [run.name for run in root.runs]

In [ ]:
out_dir = Path("cryosam_daniel_study20_outputs")

n_run_ids = len(run_ids)
n_procs = 4
for _iz in range(0, n_run_ids, n_procs):
    processes = []
    for _in in range(n_procs):
        _iz_this = _iz + _in
        if _iz_this >= n_run_ids:
            break
        run_id = run_ids[_iz_this]
        # run_id = runs_with_lysosomes[_iz_this].name
        p = mp.Process(
            target=run_in_separate_process,
            args=(run_id, out_dir, _in),
        )
        processes.append(p)
    for p in processes:
        p.start()

    for p in processes:
        p.join()

    for p in processes:
        p.close()

Process Process-268:
Traceback (most recent call last):
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1701885/244904649.py", line 105, in run_in_separate_process
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/automatic_mask_generator.py", line 202, in generate
    mask_data = self._generate_masks(image)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/automatic_

Using device:  1
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_10_3_Vol
Preprocessing the volume for Position_10_3_Vol
Running automatic segmentation for projection
Using device:  0
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_10_2_Vol
Preprocessing the volume for Position_10_2_Vol
Running automatic segmentation for projection


Process Process-270:
Traceback (most recent call last):
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1701885/244904649.py", line 105, in run_in_separate_process
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/automatic_mask_generator.py", line 202, in generate
    mask_data = self._generate_masks(image)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/automatic_

Using device:  3
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_10_5_Vol
Preprocessing the volume for Position_10_5_Vol
Running automatic segmentation for projection
Using device:  2
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_10_4_Vol
Preprocessing the volume for Position_10_4_Vol
Running automatic segmentation for projection
Saving the masks
Running automatic segmentation for volume


propagate in video: 100%|██████████| 94/94 [00:02<00:00, 31.59it/s]


...

Saving the masks and the volume


propagate in video: 100%|██████████| 93/93 [00:03<00:00, 30.95it/s]


Saving the masks and the volume


propagate in video: 100%|██████████| 94/94 [00:02<00:00, 31.67it/s]


Saving the masks and the volume
